In [ ]:
import warnings

warnings.filterwarnings("ignore")

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from keras.callbacks import Callback
from keras import backend as K
from keras.models import load_model
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_L2Normalization import L2Normalization
from math import ceil
import numpy as np
from termcolor import colored
from keras_ssd_loss import SSDLoss

from ssd_box_encode_decode_utils import SSDBoxEncoder, decode_y, decode_y2


from mn_model import mn_model
from face_generator import BatchGenerator
from config import path
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # choose gpu
%matplotlib inline

## 1. Set the model configuration parameters

In [ ]:
img_height = 512
img_width = 512
img_channels = 3

n_classes = 2
class_names = ["background", "face"]

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]  # anchorboxes for coco dataset
aspect_ratios = [
    [0.5, 1.0, 2.0],
    [1.0 / 3.0, 0.5, 1.0, 2.0, 3.0],
    [1.0 / 3.0, 0.5, 1.0, 2.0, 3.0],
    [1.0 / 3.0, 0.5, 1.0, 2.0, 3.0],
    [0.5, 1.0, 2.0],
    [0.5, 1.0, 2.0],
]  # The anchor box aspect ratios used in the original SSD300
two_boxes_for_ar1 = True
# Whether or not you want to limit the anchor boxes to lie entirely within the image boundaries
limit_boxes = True
variances = [0.1, 0.1, 0.2, 0.2]
# The variances by which the encoded target coordinates are scaled as in the original implementation
coords = "centroids"
# Whether the box coordinates to be used as targets for the model should be in the 'centroids' or 'minmax' format, see documentation
normalize_coords = True

det_model_path = "./models/"
train_data = "wider_train_v1.npy"
test_data = "wider_val_v1.npy"

## 2. Build or load the model
### 2.1 Create a new model and load trained Mobinet weights

In [ ]:
# 1: Build the Keras model.
K.clear_session()  # Clear previous models from memory.
model, model_layer, img_input, predictor_sizes = mn_model(
    image_size=(img_height, img_width, img_channels),
    n_classes=n_classes,
    min_scale=None,
    max_scale=None,
    scales=scales,
    aspect_ratios_global=None,
    aspect_ratios_per_layer=aspect_ratios,
    two_boxes_for_ar1=two_boxes_for_ar1,
    limit_boxes=limit_boxes,
    variances=variances,
    coords=coords,
    normalize_coords=normalize_coords,
)

# model.summary()

print("Freezing classification layers")
# Freeze layers
for layer_key in model_layer:
    if "detection" not in layer_key:
        # prefix detection to freeze layers which does not have detection
        model_layer[layer_key].trainable = False
print(colored("classification layers freezed", "green"))

# for layer in model.layers:
#   print (colored(layer.name, 'blue'))
#   print (colored(layer.trainable, 'green'))

# 2: Load some weights into the model.
print("Loading classification weights")
# classification_model = "./base_models/mobilenet_1_0_224_tf.h5"
classification_model = "./models/ssd_mobilenet_face_epoch_07_loss5.8451.h5"
model.load_weights(classification_model, by_name=True)
print(colored(("Classification weights %s loaded" % classification_model), "green"))

In [ ]:
# 3: Instantiate an optimizer and the SSD loss function and compile the model.
#Adam
base_lr = 0.001
adam = Adam(lr=base_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-6, decay = 0.0)
ssd_loss = SSDLoss(neg_pos_ratio=2, n_neg_min=0, alpha=1.0, beta = 1.0)
model.compile(optimizer=adam, loss=ssd_loss.compute_loss)

print(predictor_sizes)

### 2.2 Load a previously created model

In [ ]:
# # Load a previously model
# model_path = './models/ssd_mobilenet_face_epoch_01_loss6.0995.h5'

# # We need to create an SSDLoss object in order to pass that to the model loader.
# ssd_loss = SSDLoss(neg_pos_ratio=2, alpha=1.0)

# K.clear_session() # Clear previous models from memory.

# model = load_model(model_path, custom_objects={'AnchorBoxes': AnchorBoxes,
#                                                'L2Normalization': L2Normalization,
#                                                'compute_loss': ssd_loss.compute_loss})

## 3. Set up the data generators for the training

In [ ]:
batch_size = 32
num_epochs = 15
# Create ground truth box encoder
ssd_box_encoder = SSDBoxEncoder(
    img_height=img_height,
    img_width=img_width,
    n_classes=n_classes,
    predictor_sizes=predictor_sizes,
    min_scale=None,
    max_scale=None,
    scales=scales,
    aspect_ratios_global=None,
    aspect_ratios_per_layer=aspect_ratios,
    two_boxes_for_ar1=two_boxes_for_ar1,
    limit_boxes=limit_boxes,
    variances=variances,
    pos_iou_threshold=0.5,
    neg_iou_threshold=0.2,
    coords=coords,
    normalize_coords=normalize_coords,
)

In [ ]:
train_dataset = BatchGenerator(
    images_path=train_data,
    include_classes="all",
    box_output_format=["class_id", "xmin", "xmax", "ymin", "ymax"],
)

print("TRAINING DATA")

train_dataset.parse_xml(
    annotations_path=train_data,
    image_set_path=path.DRIVE_BASE_PATH,
    image_set="None",
    classes=class_names,
    exclude_truncated=False,
    exclude_difficult=False,
    ret=False,
    debug=False,
)

train_generator = train_dataset.generate(
    batch_size=batch_size,
    train=True,
    ssd_box_encoder=ssd_box_encoder,
    equalize=True,
    brightness=(0.5, 2, 0.5),
    flip=0.5,
    translate=((0, 20), (0, 30), 0.5),
    scale=(0.75, 1.2, 0.5),
    crop=False,
    # random_crop = (img_height,img_width,1,3),
    random_crop=False,
    resize=(img_height, img_width),
    # resize=False,
    gray=False,
    limit_boxes=True,
    include_thresh=0.4,
    diagnostics=False,
)

n_train_samples = train_dataset.get_n_samples()

print("Total number of training samples = {}".format(n_train_samples))

In [ ]:
print("VALIDATION DATA")

val_dataset = BatchGenerator(
    images_path=test_data,
    include_classes="all",
    box_output_format=["class_id", "xmin", "xmax", "ymin", "ymax"],
)


val_dataset.parse_xml(
    annotations_path=test_data,
    image_set_path=path.DRIVE_BASE_PATH,
    image_set="None",
    classes=class_names,
    exclude_truncated=False,
    exclude_difficult=False,
    ret=False,
    debug=False,
)


val_generator = val_dataset.generate(
    batch_size=batch_size,
    train=True,
    ssd_box_encoder=ssd_box_encoder,
    equalize=False,
    brightness=False,
    flip=False,
    translate=False,
    scale=False,
    crop=False,
    # random_crop = (img_height,img_width,1,3),
    random_crop=False,
    resize=(img_height, img_width),
    # resize=False,
    gray=False,
    limit_boxes=True,
    include_thresh=0.4,
    diagnostics=False,
)

n_val_samples = val_dataset.get_n_samples()

print("Total number of validation samples = {}".format(n_val_samples))

## 4. Set the remaining training parameters

In [ ]:
def scheduler(epoch):
    if epoch % 10 == 0 and epoch != 0:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.95)
        print("lr changed to {}".format(lr * 0.95))
    else:
        print("lr remains {}".format(K.get_value(model.optimizer.lr)))

    return K.get_value(model.optimizer.lr)


# Define a learning rate schedule.
lr_schedule = LearningRateScheduler(scheduler)

# Define model callbacks.
early_stopping = EarlyStopping(monitor="val_loss", min_delta=0.001, patience=100)
model_checkpoint = ModelCheckpoint(
    det_model_path + "ssd_mobilenet_face_epoch_{epoch:02d}_loss{val_loss:.4f}.h5",
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
    period=1,
)

## 5. Train

In [ ]:
# Now start the training
# If you're resuming a previous training, set `initial_epoch` and `final_epoch` accordingly.
initial_epoch = 0
final_epoch = 15
steps_per_epoch = ceil(n_train_samples / batch_size) * 2

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=final_epoch,
    callbacks=[model_checkpoint, lr_schedule, early_stopping],
    validation_data=val_generator,
    validation_steps=ceil(n_val_samples / batch_size),
    initial_epoch=initial_epoch,
)

## 6. Make predictions

In [ ]:
model_path = "./models/"
# model_name = "ssd_mobilenet_face_epoch_25_loss0.0916.h5"
model_name = 'ssd_mobilenet_face_epoch_30_loss0.0767.h5'

model.load_weights(model_path + model_name, by_name=True)

print(colored("weights %s loaded" % (model_path + model_name), "green"))


def save_bb(save_path, filename, results, prediction=True):

    # print filename

    img = image.load_img(filename, target_size=(img_height, img_width))
    img = image.img_to_array(img)

    filename = filename.split("/")[-1]

    if not prediction:
        filename = filename[:-4] + "_gt" + ".jpg"

    # fig,current_axis = plt.subplots(1)
    current_axis = plt.gca()

    # Get detections with confidence higher than 0.6.
    colors = plt.cm.hsv(np.linspace(0, 1, 25)).tolist()
    color_code = min(len(results), 16)
    print(colored("total number of bbs: %d" % len(results), "yellow"))
    det_conf = None
    for result in results:
        # Parse the outputs.

        if prediction:
            det_label = result[0]
            det_conf = result[1]
            det_xmin = result[2]
            det_xmax = result[3]
            det_ymin = result[4]
            det_ymax = result[5]
        else:
            det_label = result[0]
            det_xmin = result[1]
            det_xmax = result[2]
            det_ymin = result[3]
            det_ymax = result[4]

        xmin = int(det_xmin)
        ymin = int(det_ymin)
        xmax = int(det_xmax)
        ymax = int(det_ymax)

        if prediction:
            score = det_conf

        plt.imshow(img / 255.0)

        label = int(int(det_label))
        label_name = class_names[label]
        # print label_name
        # print label

        if prediction:
            display_txt = "{:0.2f}".format(score)
        else:
            display_txt = "{}".format(label_name)

        # print (xmin, ymin, ymin, ymax)
        box_coords = (xmin, ymin), (xmax - xmin), (ymax - ymin)
        color_code = color_code - 1
        color = colors[color_code]
        current_axis.add_patch(
            plt.Rectangle(*box_coords, fill=False, edgecolor=color, linewidth=2)
        )
        current_axis.text(
            xmin, ymin, display_txt, bbox={"facecolor": color, "alpha": 0.2}
        )

    current_axis.axes.get_yaxis().set_visible(False)
    current_axis.axes.get_xaxis().set_visible(False)
    plt.savefig(save_path + filename, bbox_inches="tight")

    print("saved", save_path + filename)

    plt.clf()

In [ ]:
from keras.preprocessing import image
from matplotlib import pyplot as plt

# 1: Set the generator for the predictions.
eval_dir = '/content/Object-Detection-Metrics/face/'
test_size = 100
test_generator = val_dataset.generate(
    batch_size=test_size,
    train=False,
    ssd_box_encoder=ssd_box_encoder,
    equalize=False,
    brightness=False,
    flip=False,
    translate=False,
    scale=False,
    crop=False,
    #random_crop = (img_height,img_width,1,3), 
    random_crop=False, 
    resize=(img_height, img_width), 
    #resize=False,
    gray=False,
    limit_boxes=True,
    include_thresh=0.4,
    diagnostics=False
)

print(colored("Done.", "green"))

print(colored("Now predicting...", "yellow"))

_CONF = 0.40 
_IOU = 0.1

for i in range(test_size):
    X, y, filenames = next(test_generator)
    y_pred = model.predict(X)
    # 4: Decode the raw predictions in `y_pred`.
    y_pred_decoded = decode_y2(
        y_pred,
        confidence_thresh=_CONF,
        iou_threshold=_IOU,
        top_k="all",
        normalize_coords=normalize_coords,
        input_coords=coords,
        img_height=img_height,
        img_width=img_width,
    )

    np.set_printoptions(suppress=True)

    save_bb("./output_test/", filenames[i], y_pred_decoded[i])
    save_bb("./output_test/", filenames[i], y[i], prediction=False)